In [2]:
import datetime
import glob
import os
import pymsteams
import shutil
import pyodbc
import pandas as pd



server = '192.168.1.241,1444' 
database = 'HTAUTO' 
username = 'sa' 
password = 'j^d0Okem8BXfz%2MI$sWHWAk' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cnxn.autocommit = True
cursor = cnxn.cursor()

cursor.execute('USE HTAUTO')
cursor.execute("EXEC sp_sp_start_job_wait 'UpdateBravoTemp'")

while cursor.nextset():
    pass




In [9]:
#Define các biến ban đầu
yesterday = datetime.date.today() - datetime.timedelta(days=1)
file_pattern = '*{}_{:02d}_{:02d} 17*.BAK'.format(yesterday.year, yesterday.month, yesterday.day)
HTBackUpPath = 'D:/Code linh tinh/Auto Update Data/Pull data from onedrive and update/'
HTOneDrivePath = 'C:/Users/PC/OneDrive - CONG TY TNHH HTAUTO VIET NAM (1)/BAK_bravo/'



numberOfRequiredFile = 2
listOneDrivePath = glob.glob(HTOneDrivePath + file_pattern)
listOneDriveFile = [os.path.basename(x) for x in listOneDrivePath]

listHTBackUpPath = glob.glob(HTBackUpPath + file_pattern)
listHTBackUpFile = [os.path.basename(x) for x in listHTBackUpPath]

listOfRestoreDataBase = ['B8_ASP_VN', 'B8_ASP_VN_Log']

warningWebHookURL = 'https://htautovietnam.webhook.office.com/webhookb2/96106e17-e26a-42cc-a5f2-77a6f3cc98d1@a83657d0-03aa-4e23-b7b8-27d218b177a8/IncomingWebhook/38e1f18bb8aa498487b125150289cd1d/95700413-d3f7-4ae2-a9c5-b2a613f29d96'
notificationWebHookURL = 'https://htautovietnam.webhook.office.com/webhookb2/96106e17-e26a-42cc-a5f2-77a6f3cc98d1@a83657d0-03aa-4e23-b7b8-27d218b177a8/IncomingWebhook/400ef5db810540cdb3399208c1cb2ecf/95700413-d3f7-4ae2-a9c5-b2a613f29d96'



In [6]:
#Check xem đã đủ số lượng file chưa
a=0
while len(listOneDrivePath) != numberOfRequiredFile:
    listOneDrivePath = glob.glob(HTOneDrivePath + file_pattern)
    listOneDriveFile = [os.path.basename(x) for x in listOneDrivePath]
    if a==0:
        myTeamsMessage = pymsteams.connectorcard(warningWebHookURL)
        myTeamsMessage.text("Số lượng file trong thư mục onedrive hiện đang không khớp ghi nhận: {} so với {} file <br><br> Danh sách các file trong thư mục onedrive hiện có {} <br><br> Yêu cầu bổ sung thêm".format(len(listOneDrivePath), numberOfRequiredFile, str(listOneDriveFile)))
        myTeamsMessage.send()
        a = a + 1


if len(listOneDriveFile) == numberOfRequiredFile:
    for i in range(0, numberOfRequiredFile):
        shutil.move(listOneDrivePath[i], HTBackUpPath + listOneDriveFile[i])

myTeamsMessage = pymsteams.connectorcard(notificationWebHookURL)
myTeamsMessage.text("Đã kéo {} files <br><br> Danh sách các file: {}".format(len(listHTBackUpFile), listHTBackUpFile))
myTeamsMessage.send()
        


True

In [25]:
#Restore database
for i in listHTBackUpFile:
    cursor = cnxn.cursor()
    cursor.execute('''
    USE MASTER
    RESTORE DATABASE {} FROM DISK = N'C:\BakFile\{}' with replace, recovery
    '''.format(i[:i.find('_{}'.format(yesterday.year))], i))
    while cursor.nextset():
        pass

    myTeamsMessage = pymsteams.connectorcard(notificationWebHookURL)
    myTeamsMessage.text("Đã restore Database {} từ file: {}".format(i[:i.find('_{}'.format(yesterday.year))], i))
    myTeamsMessage.send()



In [28]:
#Xóa tất cả các file BAK trong folder đó
listAllOneDriveFile = glob.glob(HTOneDrivePath + '*.BAK')
for i in listAllOneDriveFile:
    os.remove(i)

In [12]:
today = datetime.date.today()
query  = '''
select j.name
    ,js.step_name
    ,jh.sql_severity
    ,jh.message
    ,jh.run_date
    ,jh.run_time
FROM msdb.dbo.sysjobs AS j
INNER JOIN msdb.dbo.sysjobsteps AS js
   ON js.job_id = j.job_id
INNER JOIN msdb.dbo.sysjobhistory AS jh
   ON jh.job_id = j.job_id AND jh.step_id = js.step_id
WHERE run_date = '{}{:02d}{:02d}' and name = 'UpdateBravoTemp'
ORDER BY run_time DESC 
'''.format(today.year, today.month, today.day)

sql_query = pd.read_sql_query (query, cnxn)
sql_query



# myTeamsMessage = pymsteams.connectorcard(notificationWebHookURL)
# myTeamsMessage.text(sql_query.to_html())
# myTeamsMessage.send()

c:\Users\PC\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,step_name,sql_severity,message,run_date,run_time
0,UpdateBravoTemp,Other query,0,Executed as user: NT SERVICE\SQLSERVERAGENT. T...,20221206,95944
1,UpdateBravoTemp,RM detail,0,Executed as user: NT SERVICE\SQLSERVERAGENT. T...,20221206,95921
2,UpdateBravoTemp,RM by year,0,Executed as user: NT SERVICE\SQLSERVERAGENT. T...,20221206,95905
3,UpdateBravoTemp,Bảng vốn và tài sản cân đối KT,0,Executed as user: NT SERVICE\SQLSERVERAGENT. T...,20221206,95430
4,UpdateBravoTemp,Kết quả kinh doanh HT,0,Executed as user: NT SERVICE\SQLSERVERAGENT. T...,20221206,94713
5,UpdateBravoTemp,Sổ theo dõi đơn hàng đặt mua,0,Executed as user: NT SERVICE\SQLSERVERAGENT. C...,20221206,94507
6,UpdateBravoTemp,Bảng giá bán tồn kho,0,Executed as user: NT SERVICE\SQLSERVERAGENT. W...,20221206,94328
7,UpdateBravoTemp,Chi tiết chứng từ nhập,0,Executed as user: NT SERVICE\SQLSERVERAGENT. T...,20221206,93627
8,UpdateBravoTemp,Bảng kê chi phí theo khoản mục new,0,Executed as user: NT SERVICE\SQLSERVERAGENT. T...,20221206,93617
9,UpdateBravoTemp,Tồn kho HT new,0,Executed as user: NT SERVICE\SQLSERVERAGENT. I...,20221206,93553
